<img src='https://repository-images.githubusercontent.com/121802384/c355bb80-7d42-11e9-9e0e-4729609f9fbc' alt='WRF-Hydro Logo' width="15%"/>

# Lesson 4 - Run-time options for Gridded configuration

## Overview
In [Lesson 2](Lesson-2-run.ipynb) we covered the basics of constructing and running a WRF-Hydro simulation using a prepared domain for the 'Gridded' configuration. And in [Lesson 3](Lesson-3-visualize.ipynb), we covered how to do some basic plotting for the 2D input/output files and also plot some time series. 

In this lesson, we experiment with few different run-time options and do some basic visualization of the outputs using the Python library *xarray*.  

**NOTE: If you have not completed Lessons 2 and 3, please stop and do so now.**

## Recap of Lessons 1 and 2, compiling and running WRF-Hydro
In this section, we quickly recap the commands issued in [Lessons 1](Lesson-1-compile.ipynb) and [2](Lesson-2-run.ipynb) to compile WRF-Hydro and create our simulation directory. We use the prepared domain for the remaining lessons, but feel free to replace the prepared domain with the domain you create in the geospatial hands-on training session. 

**NOTE: Your domain directory structure and filenames must match the prepared domain EXACTLY**

### Compiling WRF-Hydro
Below are the commands to compile WRF-Hydro.

In [ ]:
%%bash
# Change to the trunk/NDHMS directory and configure for GNU gfortran
cd ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS
./configure 2

# Make a copy of the template environment variable file, setEnvars.sh
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/template/setEnvar.sh \
~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/setEnvar.sh

**Set compile-time options**

Edit the `setEnvar.sh` script to set environment variables required by the compile script. Your [setEnvar.sh](../wrf_hydro_nwm_public/trunk/NDHMS/setEnvar.sh) should look like the bash script below when you are finished. 

**NOTE:** We are compiling with spatial soil active for this exercise.

---

```bash
#!/bin/bash
# WRF-Hydro compile time options

# This is a WRF environment variable. Always set to 1=On for compiling WRF-Hydro.
export WRF_HYDRO=1

# Enhanced diagnostic output for debugging: 0=Off, 1=On.
export HYDRO_D=1

# Spatially distributed parameters for NoahMP: 0=Off, 1=On.
export SPATIAL_SOIL=1  

# RAPID model: 0=Off, 1=On.
export WRF_HYDRO_RAPID=0

# WCOSS file units: 0=Off, 1=On. 
export NCEP_WCOSS=0

# NWM output metadata: 0=Off, 1=On.
export NWM_META=0

# Streamflow nudging: 0=Off, 1=On.
export WRF_HYDRO_NUDGING=0
```

---


In [ ]:
%%bash
# Edit setEnvar.sh to turn on debug (HYDRO_D=1) and spatial soils (SPATIAL_SOIL=1)
sed -i -e 's#export HYDRO_D=0#export HYDRO_D=1#g' ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/setEnvar.sh
sed -i -e 's#export SPATIAL_SOIL=0#export SPATIAL_SOIL=1#g' ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/setEnvar.sh

cat ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/setEnvar.sh

**Compile WRF-Hydro in standalone mode**

In [ ]:
%%bash
cd ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS
./compile_offline_NoahMP.sh setEnvar.sh >> compile.log 2>&1

tail -13 ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/compile.log

**We have successfully compiled WRF-Hydro and we will use this binary and *.TBL files for the remaining simulations in this lesson** 
Let 's edit the SWE_LIMIT in the MPATABLE.TBL file. 

In [ ]:
%%bash
# Set the SWE Limit to a very high number
sed -i -e 's#SWE_LIMIT = 5000.0#SWE_LIMIT = 5000000.0#g' ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run/MPTABLE.TBL

### Creating the simulation directory and running a baseline WRF-Hydro simulation
Below are the commands from Lesson 2 to create the simulation directory and run a WRF-Hydro simulation. This simulation will be run exactly as was done in Lesson 2 and will serve as our baseline simulation. We will modify run-time options in subsequent simulations in the lesson and compare the outputs.

In [ ]:
%%bash
# Make a new directory for our baseline simulation
mkdir -p ~/wrf-hydro-training/output/lesson4/run_gridded_baseline

# Copy our model files to the simulation directory
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run/*.TBL \
~/wrf-hydro-training/output/lesson4/run_gridded_baseline
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run/wrf_hydro.exe \
~/wrf-hydro-training/output/lesson4/run_gridded_baseline

# Create symbolic links to large domain files
cp -as $HOME/wrf-hydro-training/example_case/FORCING \
~/wrf-hydro-training/output/lesson4/run_gridded_baseline
cp -as $HOME/wrf-hydro-training/example_case/Gridded/DOMAIN \
~/wrf-hydro-training/output/lesson4/run_gridded_baseline
cp -as $HOME/wrf-hydro-training/example_case/Gridded/RESTART \
~/wrf-hydro-training/output/lesson4/run_gridded_baseline

# Copy namelist files
cp ~/wrf-hydro-training/example_case/Gridded/namelist.hrldas \
~/wrf-hydro-training/output/lesson4/run_gridded_baseline
cp ~/wrf-hydro-training/example_case/Gridded/hydro.namelist \
~/wrf-hydro-training/output/lesson4/run_gridded_baseline

# Run the simulation
cd ~/wrf-hydro-training/output/lesson4/run_gridded_baseline
mpirun -np 2 ./wrf_hydro.exe >> run.log 2>&1

In [ ]:
%%bash
# Check that it finished successfully
tail -1 ~/wrf-hydro-training/output/lesson4/run_gridded_baseline/diag_hydro.00000

## Experimenting with run-time options
### Setup
In this section we will run a few different simulations with different run-time options and explore their effects on the model output.

We will create a new simulation directory for each experiment. First, we will make a new simulation directory as we did with the baseline run above and use this as a template for creating multiple new simulation directories.

**Step 1: Create a simulation directory to use as a template**

In [ ]:
%%bash
# Make a new directory for our baseline simulation
mkdir -p ~/wrf-hydro-training/output/lesson4/run_gridded_template

# Copy our model files to the simulation directory
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run/*.TBL \
~/wrf-hydro-training/output/lesson4/run_gridded_template
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run/wrf_hydro.exe \
~/wrf-hydro-training/output/lesson4/run_gridded_template

# Create symbolic links to large domain files
cp -as $HOME/wrf-hydro-training/example_case/FORCING \
~/wrf-hydro-training/output/lesson4/run_gridded_template
cp -as $HOME/wrf-hydro-training/example_case/Gridded/DOMAIN \
~/wrf-hydro-training/output/lesson4/run_gridded_template
cp -as $HOME/wrf-hydro-training/example_case/Gridded/RESTART \
~/wrf-hydro-training/output/lesson4/run_gridded_template

# Copy namelist files
cp ~/wrf-hydro-training/example_case/Gridded/namelist.hrldas \
~/wrf-hydro-training/output/lesson4/run_gridded_template
cp ~/wrf-hydro-training/example_case/Gridded/hydro.namelist \
~/wrf-hydro-training/output/lesson4/run_gridded_template

**Step 2: View the contents**

In [ ]:
%%bash
ls ~/wrf-hydro-training/output/lesson4/run_gridded_template

We now have our template simulation directory ready to go. In each experiment we will copy this directory and edit the two namelist files *namelist.hrldas* and *hydro.namelist* to change run-time options.

## Experiment 1 - Warm vs cold start
### Background
A model **cold start** is when you start your simulation with default initial conditions. In this case, model states are initialized from the `wrfinput.nc` file (land surface states) and `GWBUCKPARM.nc` (groundwater bucket states, if active). Often these initial values are guesses at global, physically plausible starting values. However, model states like soil moisture, groundwater levels, and stream levels need time to adjust to local climate and environment. So in most cases, we first **spinup** the model to reach an equilibrium state before launching a model experiment and evaluating output. Model states at the end of this **spinup** period can be used to **warm start** a simulation. A **warm start** is when the model simulation begins with the simulated values from a previous run. How long you need to **spinup** your model varies based on the local climate (e.g., drier climates generally take longer than wetter climates), the variables you are interested in (e.g., a fast stabilizing state like shallow soil moisture vs. a slower evolving state like deep groundwater), and local topography and soil types (e.g., sands have shorter memory than clays). 

### Objective
The goal of this experiment is to demonstrate the effect of starting a simulation from a *cold stat* vs. a *warm start*. The Gridded baseline run was started from a *warm start*, so all we need to do is construct a simulation directory and edit the namelists to start from a cold start.

### Create experiment simulation directory
We will copy our template directory `~/wrf-hydro-training/output/lesson4/run_gridded_template` and edit the `namelist.hrldas` and `hydro.namelist` files to start the model from a cold start. 

In [ ]:
%%bash
cp -r ~/wrf-hydro-training/output/lesson4/run_gridded_template ~/wrf-hydro-training/output/lesson4/run_cold_start

### Edit the namelists and run the simulation
If restart files are not provided in the `namelist.hrldas` and `hydro.namelist` files then the simulation will start from a cold start. 

**Step 1: Edit the *namelist.hrldas* file**

Comment out this line `RESTART_FILENAME_REQUESTED = "RESTART/RESTART.2011082600_DOMAIN1"` in the [namelist.hrldas](../output/lesson4/run_cold_start/namelist.hrldas) by adding `!` to the start of the line. 

```
!RESTART_FILENAME_REQUESTED = "RESTART/RESTART.2011082600_DOMAIN1"
```

**Step 2: Edit the *hydro.namelist* file**

Comment out this line `RESTART_FILE  = 'RESTART/HYDRO_RST.2011-08-26_00_00_DOMAIN1'` in the [hydro.namelist](../output/lesson4/run_cold_start/hydro.namelist) by adding `!` to the start of the line. 

```
! Specify the name of the restart file if starting from restart...comment out with '!' if not...
!RESTART_FILE  = 'RESTART/HYDRO_RST.2011-08-26_00_00_DOMAIN1'
```
Tell the code to use GWBUCKPARM.nc initialization values vs. the restart file starting states by editing the `GW_RESTART` option in the *hydro.namelist*. 

```
! Specify baseflow/bucket model initialization...(0=cold start from table, 1=restart file)
GW_RESTART = 0
```


In [ ]:
%%bash
# Edit setEnvar.sh to turn on debug (HYDRO_D=1) and spatial soils (SPATIAL_SOIL=1)
sed -i -e 's#RESTART_FILENAME_REQUESTED#!RESTART_FILENAME_REQUESTED#g'  ~/wrf-hydro-training/output/lesson4/run_cold_start/namelist.hrldas
sed -i -e 's#RESTART_FILE#!RESTART_FILE#g'  ~/wrf-hydro-training/output/lesson4/run_cold_start/hydro.namelist
sed -i -e 's#GW_RESTART = 1#GW_RESTART = 0#g'  ~/wrf-hydro-training/output/lesson4/run_cold_start/hydro.namelist

**Step 3: Run the simulation**

In [ ]:
%%bash
cd ~/wrf-hydro-training/output/lesson4/run_cold_start
mpirun -np 2 ./wrf_hydro.exe >> run.log 2>&1

**Step 4: Check that the simulation finished successfully**

In [ ]:
%%bash
tail -1 ~/wrf-hydro-training/output/lesson4/run_cold_start/diag_hydro.00000

### Results
We will now look at the differences in streamflow between our warm start simulation represented by the *run_gridded_baseline* simulation and our cold start simulation that we ran during this experiment.

We will use Python and the `xarray` library to create hydrographs and 2d plots of soil moisture. For an intro to these tools, please see [Lesson 3](Lesson-3-visualize.ipynb).

**Load the xarray python package**

In [ ]:
# Load required packages
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt

xr.set_options(display_style="html")
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

#### Hydrographs
**Load the CHANOBS streamflow datasets**

We are going to use the *CHANOBS* output files to extract the streamflow time series. *CHANOBS* output files contain information for only the grid cells where we have specified a gage is located.

In [ ]:
chanobs_baseline = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson4/run_gridded_baseline/*CHANOBS*',
                            combine='by_coords')
chanobs_cold = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson4/run_cold_start/*CHANOBS*',
                            combine='by_coords')
obs = pd.read_csv('/home/docker/wrf-hydro-training/example_case/streamflow_obs.csv',dtype=str)
obs['dateTime'] = pd.to_datetime(obs['dateTime'])
obs['streamflow_cms'] = pd.to_numeric(obs['streamflow_cms'])

**Plot the hydrographs**

**Table 1. Stream Gage IDs and associated WRF-Hydro feature_id indices.**

|Gage ID|WRF-Hydro gage feature_id|
|------|:-:|
| 19372 | 6 |

In [ ]:
fig, axes = plt.subplots(ncols=1,figsize=(12, 6))
plt.suptitle('Hydrographs for warm and cold starts',fontsize=24)
chanobs_baseline.sel(feature_id = 6).streamflow.plot(label='Warm start',
                                                     color='red',
                                                     linestyle='--')
chanobs_cold.sel(feature_id = 6).streamflow.plot(label='Cold start',
                                                        color='blue',
                                                        linestyle='dotted')

obs[obs['site_no'] == '19372'].plot(x='dateTime',
                                       y='streamflow_cms',
                                       ax=axes,
                                       label='Observed',
                                       color='gray')
plot_times = chanobs_baseline.time.data
plot_time_buffer = pd.to_timedelta(2, unit='D') 
plt.xlim([plot_times.min()-plot_time_buffer, plot_times.max()+plot_time_buffer])
plt.legend()
plt.show()

#### Soil moisture at start of simulation
The option `t0OutputFlag` in the `hydro.namelist` can be used to output models states at the start of the simulation.

```
! Option to write output files at time 0 (restart cold start time): 0=no, 1=yes (default)
t0OutputFlag = 1
```
We had this option set to 1 for both of our simulations so that we have outputs at the first timestep. 

**Load the LDASOUT datasets from the land surface model**

In [ ]:
ldasout_baseline = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson4/run_gridded_baseline/*LDASOUT*',
                            combine='by_coords')
ldasout_cold = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson4/run_cold_start/*LDASOUT*',
                            combine='by_coords')

**Plot soil moisture at start of simulation**

In [ ]:
# Select data for t=0 at surface layer
ldasout_baseline_t0 = ldasout_baseline.isel(time = 0).sel(soil_layers_stag = 0)
ldasout_cold_t0 = ldasout_cold.isel(time = 0).sel(soil_layers_stag = 0)

fig, axes = plt.subplots(ncols=2,figsize=(15, 6))
plt.subplots_adjust(wspace=.5)
plt.suptitle('Surface soil moisture states at t=0 for warm and cold starts',fontsize=24)
ldasout_baseline_t0.SOIL_M.plot(ax=axes[0],vmin=0.1,vmax=0.5)
axes[0].set_title('Warm start')
ldasout_cold_t0.SOIL_M.plot(ax=axes[1],vmin=0.1,vmax=0.5)
axes[1].set_title('Cold start')
plt.show()

### Discussion

Note that the streamflow for the cold start at the start of the simulation is 0 while the streamflow from the warm start (baseline) is > 0. Additionally, the cold start simulation begins with relatively low soil moisture (coming from the *SMOIS* variable in the `wrfinput_d01.nc` file) while the warm start simulation begins with higher soil moisture content (coming from the restart file) over the non glacier pixels. There are other existing differneces in the initial consitions which result in different model behavoir for example having a unrealistic high flows at the initial of the cold start simulations. This is a good example of why it is recommended to start with a warm model. 

# Next up - Exploring terrain physics
This concludes lesson 4. 

In the [next lesson](Lesson-5-land-surface.ipynb), we will explore terrain physics options and parameters.

**IT IS BEST TO EITHER SHUTDOWN THIS LESSON OR CLOSE IT BEFORE PROCEEDING TO THE NEXT LESSON TO AVOID POSSIBLY EXCEEDING ALLOCATED MEMORY. Shutdown the lesson be either closing the browser tab for the lesson or selecting `Kernel -> Shut Down Kernel` in JupyterLab.**

© UCAR 2020